# Como obtener solo los vectores

In [ ]:
!pip install -U sentence-transformers --quiet

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
#Textos a vectorizar
sentences = [
    'Italy',
    'Spain',
    'Germany',
    'France'
]

In [ ]:
#Las frases son vectorizadas llamando a model.encode()
output = model.encode(sentences)

In [ ]:
embeddings = pd.DataFrame(output)

In [ ]:
embeddings

## El viejo y conocido KNN vuelve a ser util

Si utilizamos como medida de distancia entre vectores el angulo entre ellos a traves de la metrica de similitud de coseno podemos ver cuales de nuestros vectores son mas similares y con ello podemos utilizar el algoritmo de vecinos cercanos para hacer, por ejemplo, una busqueda de las cinco frases mas parecidas a una que nosotros elijamos.

In [ ]:
query = "Madrid"
query_embedding = model.encode(query)

In [ ]:
for i in range(len(embeddings)):
    cos_sim = util.cos_sim(query_embedding, embeddings.iloc[i,:])
    print(f"Similitud del coseno {i}: {cos_sim} para {sentences[i]}")

### Ejercicio 1:

Descarga un chat de whatsapp y utiliza los vectores y la similitud para buscar textos en dicho chat. Por ejemplo, supongamos que queremos buscar una receta de cocina de pasta y tenemos el siguiente chat:
- Persona A: "Hola como estas?"
- Persona B: "Buen dia!"
- Persona B: "Muy bien y tu?"
- Persona A: "Bien, con un poco de calor"
- Persona A: "Queria hacerte una consulta a ti que eres experto en la cocina"
- Persona B: "Si dime que quieres saber"
- Persona A: "Quisiera saber cual es tu receta para hacer tallarines"
- Persona B: "Si por supuesto!"
- Persona B: "Es super simple: 1 huevo y 100 gramos de harina por persona"
- Persona B: "Si que es facil verdad?"
- Persona A: "Si! Muchas gracias!!"

Usar como query la siguiente frase: "como haces tu pasta"

Analizar luego que pasa con la busqueda de las siguientes frases:
- "que ingredientes lleva la receta"
- "Tallarines"


In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 1: Paso 1 {display-mode:"form"}

# Cargar un modelo en espanol
model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-cased')

In [3]:
#@title Solucion Ejercicio 1: Paso 2 {display-mode:"form"}

# Textos a vectorizar
chats = [
"Hola como estas?",
"Buen dia!",
"Muy bien y tu?",
"Bien, con un poco de calor",
"Queria hacerte una consulta a ti que eres experto en la cocina",
"Si dime que quieres saber",
"Quisiera saber cual es tu receta para hacer tallarines",
"Si por supuesto!",
"Es super simple: 1 huevo y 100 gramos de harina por persona",
"Si que es facil verdad?",
"Si! Muchas gracias!!"
]

In [ ]:
#@title Solucion Ejercicio 1: Paso 3 {display-mode:"form"}

# Convertir los textos a vectores
vectores = model.encode(chats)
vectores

In [ ]:
#@title Solucion Ejercicio 1: Paso 4 {display-mode:"form"}

# Convertir el texto a buscar en vector
#vector_a_buscar = model.encode("como haces tu pasta")
#vector_a_buscar = model.encode("que ingredientes lleva la receta")
vector_a_buscar = model.encode("Tallarines")

In [ ]:
#@title Solucion Ejercicio 1: Paso 5 {display-mode:"form"}
similitud_chats = []
# Calcular que vector del chat es mas similar al vector a buscar
for i in range(len(vectores)):
    cos_sim = util.cos_sim(vector_a_buscar, vectores[i,:])
    similitud_chats.append((float(cos_sim), chats[i]))

similitud_chats.sort(key=lambda tup: -tup[0])

for similitud, texto in similitud_chats:
    print(f"{similitud:0.2f}: {texto}")

### Ejercicio 1.2:

Realizar el mismo ejercicio pero utlizando una base de datos vectorial como chromadb


In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 1.2: Paso 1 {display-mode:"form"}

!pip install chromadb -U -q
!pip install sentence-transformers -U -q

In [ ]:
#@title Solucion Ejercicio 1.2: Paso 2 {display-mode:"form"}

import pandas as pd
from sentence_transformers import SentenceTransformer, util
import chromadb

In [ ]:
#@title Solucion Ejercicio 1.2: Paso 3 {display-mode:"form"}

# Cargar un modelo en espanol
model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-cased')

# Textos a vectorizar
chats = [
"Hola como estas?",
"Buen dia!",
"Muy bien y tu?",
"Bien, con un poco de calor",
"Queria hacerte una consulta a ti que eres experto en la cocina",
"Si dime que quieres saber",
"Quisiera saber cual es tu receta para hacer tallarines",
"Si por supuesto!",
"Es super simple: 1 huevo y 100 gramos de harina por persona",
"Si que es facil verdad?",
"Si! Muchas gracias!!"
]

# Convertir los textos a vectores
vectores = model.encode(chats)

# Crear una base de datos Chroma
client = chromadb.Client()
collection = client.create_collection("whatsapp_chats")

# Agregar los vectores y textos a la colección
collection.add(
    documents=chats,
    embeddings=vectores.tolist(),
    ids=[str(i) for i in range(len(chats))],
)

In [ ]:
#@title Solucion Ejercicio 1.2: Paso 4 {display-mode:"form"}

# Función para realizar la búsqueda
def search_in_chats(query):
  query_embedding = model.encode(query)
  results = collection.query(
      query_embeddings=query_embedding.tolist(),
      n_results=len(chats)
  )
  return results

# Realizar la búsqueda para diferentes queries
queries = [
    "como haces tu pasta",
    "que ingredientes lleva la receta",
    "Tallarines"
]

for query in queries:
  results = search_in_chats(query)
  print(f"Resultados para la query: '{query}'")
  for i, (document, distance) in enumerate(zip(results['documents'][0], results['distances'][0])):
    print(f"  {i+1}. {document} (Similitud: {1-distance:.2f})")
  print("\n")

### Ejercicio 2:

Realizar el mismo ejercicio pero utlizando la busqueda de full text con el algoritmo bm25.



In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 2: Paso 1 {display-mode:"form"}
!pip install bm25s -U -q

In [ ]:
#@title Solucion Ejercicio 2: Paso 2 {display-mode:"form"}
import bm25s

In [ ]:
#@title Solucion Ejercicio 2: Paso 3 {display-mode:"form"}

# Create the BM25 model and index the corpus
retriever = bm25s.BM25(corpus=chats)
retriever.index(bm25s.tokenize(chats))

In [ ]:
#@title Solucion Ejercicio 2: Paso 4 {display-mode:"form"}

# Query the corpus and get top-k results
#query = "como haces tu pasta"
#query = "que ingredientes lleva la receta"
query = "Tallarines"
results, scores = retriever.retrieve(bm25s.tokenize(query), k=2)

In [ ]:
#@title Solucion Ejercicio 2: Paso 5 {display-mode:"form"}

print(results[0, 0], ":", scores[0, 0])
print(results[0, 1], ":", scores[0, 1])

## Vectores de otras APIs

- [OpenAI](https://platform.openai.com/docs/guides/embeddings/use-cases)
- [Ollama](https://github.com/ollama/ollama/blob/main/docs/api.md#generate-embeddings)

### Referencias:
- [Sbert](https://www.sbert.net/)
- [BM25 en python](https://huggingface.co/blog/xhluca/bm25s)
- [ChromaDB](https://docs.trychroma.com/getting-started)
- Imagenes y texto con CLIP:
    - [Usando sentence bert para busqueda de imagenes](https://www.sbert.net/examples/applications/image-search/README.html)
    - [Notebook de ejemplo](https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/image-search/Image_Search.ipynb)
- [Customizing Embeddings](https://cookbook.openai.com/examples/customizing_embeddings)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)